In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical'}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops','contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v
/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

/home/smb/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_false,hit_true,hit_rate,num_stops,search_conducted_false_search_conducted_true_dpgmm,search_conducted_false_contraband_found_false_dpgmm,search_conducted_false_num_stops_dpgmm,search_conducted_true_contraband_found_false_dpgmm,search_conducted_true_num_stops_dpgmm,contraband_found_false_num_stops_dpgmm
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,1.0,NaN,NaN,31.0,0,1,1,0,0,2
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,1.0,0.5,3.0,0,1,1,0,0,2
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,1.0,NaN,NaN,15.0,0,1,1,0,0,2
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,2.0,0.5,36.0,0,1,1,0,0,2
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,3.0,NaN,NaN,61.0,0,1,1,0,0,2


In [11]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


In [12]:
labeled_df.add_quantile(['hit_rate','num_stops'])

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_rate,num_stops,search_conducted_false_search_conducted_true_dpgmm,search_conducted_false_contraband_found_false_dpgmm,search_conducted_false_num_stops_dpgmm,search_conducted_true_contraband_found_false_dpgmm,search_conducted_true_num_stops_dpgmm,contraband_found_false_num_stops_dpgmm,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,31.0,0,1,1,0,0,2,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,0.500000,3.0,0,1,1,0,0,2,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,15.0,0,1,1,0,0,2,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,0.500000,36.0,0,1,1,0,0,2,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,61.0,0,1,1,0,0,2,high,low
5,AZ,2010,F,Black,0,1,0,0,224.0,8.0,...,NaN,232.0,0,1,1,0,0,2,high,low
6,AZ,2010,F,Hispanic,0,1,0,0,557.0,33.0,...,0.030303,590.0,0,1,1,0,0,2,low,low
7,AZ,2010,F,Other,0,1,0,0,167.0,6.0,...,NaN,173.0,0,1,1,0,0,2,high,low
8,AZ,2010,F,White,0,1,0,0,3145.0,92.0,...,0.076087,3237.0,0,1,1,0,0,2,low,mid
9,AZ,2010,M,Asian,0,1,0,0,97.0,1.0,...,NaN,98.0,0,1,1,0,0,2,high,low


In [13]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,binary,[groupby],False,NaN
medical,int64,binary,[groupby],False,NaN
recreational,int64,binary,[groupby],False,NaN
no_reforms,int64,binary,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [14]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [15]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.all_linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [16]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,subgroup_trend_quality,group_feat,subgroup,trend_type,agg_trend,agg_trend_quality
0,year,search_conducted_rate,-0.247018,0.247018,state,AZ,pearson_corr,-0.03903,0.03903
1,year,search_conducted_rate,-0.414566,0.414566,state,CO,pearson_corr,-0.03903,0.03903
2,year,search_conducted_rate,0.118238,0.118238,state,CT,pearson_corr,-0.03903,0.03903
3,year,search_conducted_rate,-0.199765,0.199765,state,IL,pearson_corr,-0.03903,0.03903
4,year,search_conducted_rate,-0.603026,0.603026,state,MA,pearson_corr,-0.03903,0.03903


Now we can use a list of objects and apply multiple trends

In [17]:
labeled_df.df['num_stops'].describe()

count    1.069000e+03
mean     5.870794e+04
std      1.264399e+05
min      3.000000e+00
25%      2.120000e+03
50%      9.253000e+03
75%      5.438100e+04
max      1.237890e+06
Name: num_stops, dtype: float64

In [18]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:117: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:119: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality
1746,search_conducted_rate,num_stops,state,SC,-231200,0.057097,lin_reg,-257201,0.065594
335,contraband_found_rate,driver_race,search_conducted_false_contraband_found_false_...,6,"[White, Hispanic, Black]",0.295800,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600
208,contraband_found_rate,driver_gender,driver_race,Asian,"[F, M]",0.133200,rank_trend,"[F, M]",0.179700
317,contraband_found_rate,driver_race,search_conducted_false_search_conducted_true_d...,0,"[Asian, Other, Hispanic, White, Black]",0.235600,rank_trend,"[Asian, White, Other, Hispanic, Black]",0.071600
893,contraband_found_rate,state,search_conducted_false_search_conducted_true_d...,10,"[NC, WA, CO, SC, TX, MA]",0.717100,rank_trend,"[NC, CO, MD, WA, CT, SC, TX, WI, IL, MA, VT, A...",0.020100
1109,num_stops,state,search_conducted_true_num_stops_dpgmm,4,"[NC, TX]",0.492100,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900
478,hit_rate,driver_gender,contraband_found_false_num_stops_dpgmm,5,"[F, M]",1.000000,rank_trend,"[F, M]",0.043300
1195,year,search_conducted_rate,search_conducted_true_contraband_found_false_d...,0,-0.000335359,0.034291,lin_reg,-0.000353381,0.039030
468,hit_rate,driver_gender,search_conducted_true_num_stops_dpgmm,6,"[F, M]",0.331000,rank_trend,"[F, M]",0.043300
1820,search_conducted_rate,num_stops,search_conducted_true_num_stops_dpgmm,8,-5.66568e+06,0.670638,lin_reg,-257201,0.065594


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [19]:
pd.unique(labeled_df.result_df['trend_type'])

array(['rank_trend', 'lin_reg'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [20]:
labeled_df.trend_list

In [21]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance()

labeled_df.result_df.sample(10)

/home/smb/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3559: RuntimeWarning: invalid value encountered in double_scalars
  size * (size - 1) * (size - 2))


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
609,num_stops,driver_gender,no_reforms,0,"[F, M]",0.1080,rank_trend,"[F, M]",0.1164,-0.5000
685,num_stops,driver_race,state,CO,"[Other, Asian, Black, Hispanic, White]",0.8172,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.4996,-0.5000
500,hit_rate,driver_race,state,WA,"[Asian, Hispanic, Black, Other, White]",0.3492,rank_trend,"[Asian, Hispanic, Other, Black, White]",0.0023,-0.3000
230,contraband_found_rate,driver_gender,search_conducted_false_search_conducted_true_d...,9,[M],NaN,rank_trend,"[F, M]",0.1797,NaN
239,contraband_found_rate,driver_gender,search_conducted_false_contraband_found_false_...,6,[M],NaN,rank_trend,"[F, M]",0.1797,NaN
813,search_conducted_rate,state,search_conducted_false_contraband_found_false_...,6,"[WA, TX]",0.0286,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.3881,-0.1494
639,num_stops,driver_gender,search_conducted_false_num_stops_dpgmm,5,"[F, M]",0.1111,rank_trend,"[F, M]",0.1164,-0.5000
927,contraband_found_rate,state,search_conducted_true_contraband_found_false_d...,9,[TX],NaN,rank_trend,"[NC, CO, MD, WA, CT, SC, TX, WI, IL, MA, VT, A...",0.0201,-0.3645
762,num_stops,driver_race,search_conducted_true_num_stops_dpgmm,9,"[Other, Black, Hispanic, White, Asian]",0.1057,rank_trend,"[Other, Asian, Black, Hispanic, White]",0.4996,0.1000
1055,num_stops,state,recreational,0,"[RI, VT, WI, CT, MD, MA, AZ, IL, CO, NC, SC, W...",0.3350,rank_trend,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.3209,-0.4487


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [22]:
labeled_df.trend_list[0].trend_precompute

{'rank_trend_agg_trend_contraband_found_rate_driver_gender':                    stat       max       min
 driver_gender                              
 F              0.003202  0.005325  0.001079
 M              0.006898  0.011263  0.002532,
 'rank_trend_agg_trend_contraband_found_rate_driver_race':                  stat       max       min
 driver_race                              
 Asian        0.002658  0.004362  0.000955
 White        0.004997  0.007971  0.002022
 Other        0.005894  0.011755  0.000032
 Hispanic     0.006466  0.010447  0.002485
 Black        0.008424  0.015162  0.001686,
 'rank_trend_agg_trend_contraband_found_rate_state':            stat       max       min
 state                              
 NC     0.001065  0.001780  0.000350
 CO     0.002304  0.003695  0.000913
 MD     0.004329  0.008003  0.000656
 WA     0.004353  0.007261  0.001446
 CT     0.005764  0.008936  0.002593
 SC     0.006385  0.009862  0.002908
 TX     0.006541  0.009586  0.003496
 WI     0.0079

# Filtering

In [23]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module detect_simpsons_paradox.ranking_processing:

get_trend_rows(feat1=None, feat2=None, group_feat=None, subgroup=None) method of detect_simpsons_paradox.labeled_dataframe.labeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    feat1 : str, list, or  None
        trend variable name or None to include all
    feat2 : str, list, or  None
        trend variable name or None to include all
    group_feat : str, list, or  None
        groupoby variable name or None to include all
    subgroup : str, list, or  None
        value of groupby_feat or or None to include all



So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [24]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance
1148,year,search_conducted_rate,driver_race,Black,-0.000555503,0.039174,lin_reg,-0.000353381,0.039030,0.000129
1149,year,search_conducted_rate,driver_race,Hispanic,-0.0014828,0.177694,lin_reg,-0.000353381,0.039030,0.000719
1451,year,num_stops,driver_race,Black,-2262.72,0.169689,lin_reg,-1560.86,0.043965,0.000127
1452,year,num_stops,driver_race,Hispanic,422.667,0.014732,lin_reg,-1560.86,0.043965,0.998086


We can also filter based on SP

In [25]:
labeled_df.get_SP_rows(thresh=.2)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh0.2
34,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_true_d...,8,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,1.500000,True
36,search_conducted_rate,driver_gender,search_conducted_false_search_conducted_true_d...,10,"[M, F]",0.178600,rank_trend,"[F, M]",0.251900,1.500000,True
42,search_conducted_rate,driver_gender,search_conducted_false_contraband_found_false_...,4,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,1.500000,True
51,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,2,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,1.500000,True
53,search_conducted_rate,driver_gender,search_conducted_false_num_stops_dpgmm,4,"[M, F]",0.115900,rank_trend,"[F, M]",0.251900,1.500000,True
69,search_conducted_rate,driver_gender,search_conducted_true_contraband_found_false_d...,8,"[M, F]",0.298100,rank_trend,"[F, M]",0.251900,1.500000,True
76,search_conducted_rate,driver_gender,search_conducted_true_num_stops_dpgmm,4,"[M, F]",0.881900,rank_trend,"[F, M]",0.251900,1.500000,True
89,search_conducted_rate,driver_gender,contraband_found_false_num_stops_dpgmm,6,"[M, F]",0.379700,rank_trend,"[F, M]",0.251900,1.500000,True
176,search_conducted_rate,driver_race,search_conducted_true_num_stops_dpgmm,8,"[Hispanic, White, Black]",0.024400,rank_trend,"[Asian, White, Other, Black, Hispanic]",0.277200,0.276400,True
229,contraband_found_rate,driver_gender,search_conducted_false_search_conducted_true_d...,8,"[M, F]",0.881900,rank_trend,"[F, M]",0.179700,1.500000,True


# Ranking

In [26]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh0.2', 'mean_view_distance'],
      dtype='object')


,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh0.2,mean_view_distance
1287,year,num_stops,search_conducted_false_num_stops_dpgmm,11,28497.9,0.646655,lin_reg,-1560.86,0.043965,0.999570,True,0.43654
1257,year,num_stops,search_conducted_false_search_conducted_true_d...,1,22252.9,0.572930,lin_reg,-1560.86,0.043965,0.999564,True,0.43654
1290,year,num_stops,search_conducted_true_contraband_found_false_d...,2,22252.9,0.572930,lin_reg,-1560.86,0.043965,0.999564,True,0.43654
1272,year,num_stops,search_conducted_false_contraband_found_false_...,6,13882.7,0.424945,lin_reg,-1560.86,0.043965,0.999546,True,0.43654
1305,year,num_stops,search_conducted_true_num_stops_dpgmm,8,12793.9,0.418378,lin_reg,-1560.86,0.043965,0.999542,True,0.43654
1312,year,num_stops,contraband_found_false_num_stops_dpgmm,4,12416,1.000000,lin_reg,-1560.86,0.043965,0.999541,True,0.43654
1294,year,num_stops,search_conducted_true_contraband_found_false_d...,7,9752.58,0.311244,lin_reg,-1560.86,0.043965,0.999527,True,0.43654
1228,year,num_stops,state,AZ,8474.29,0.301916,lin_reg,-1560.86,0.043965,0.999517,True,0.43654
1261,year,num_stops,search_conducted_false_search_conducted_true_d...,6,7240.54,0.197741,lin_reg,-1560.86,0.043965,0.999504,True,0.43654
1300,year,num_stops,search_conducted_true_num_stops_dpgmm,2,6626.86,0.114329,lin_reg,-1560.86,0.043965,0.999496,True,0.43654


In [27]:
labeled_df.add_view_score('SP_thresh0.2',agg_type='sum',colored=False).head(10)

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_quality,trend_type,agg_trend,agg_trend_quality,distance,SP_thresh0.2,mean_view_distance,sum_view_SP_thresh0.2
0,year,num_stops,search_conducted_false_num_stops_dpgmm,11,28497.9,0.646655,lin_reg,-1560.86,0.043965,0.999570,True,0.43654,42.0
1,year,num_stops,search_conducted_false_search_conducted_true_d...,1,22252.9,0.572930,lin_reg,-1560.86,0.043965,0.999564,True,0.43654,42.0
2,year,num_stops,search_conducted_true_contraband_found_false_d...,2,22252.9,0.572930,lin_reg,-1560.86,0.043965,0.999564,True,0.43654,42.0
3,year,num_stops,search_conducted_false_contraband_found_false_...,6,13882.7,0.424945,lin_reg,-1560.86,0.043965,0.999546,True,0.43654,42.0
4,year,num_stops,search_conducted_true_num_stops_dpgmm,8,12793.9,0.418378,lin_reg,-1560.86,0.043965,0.999542,True,0.43654,42.0
5,year,num_stops,contraband_found_false_num_stops_dpgmm,4,12416,1.000000,lin_reg,-1560.86,0.043965,0.999541,True,0.43654,42.0
6,year,num_stops,search_conducted_true_contraband_found_false_d...,7,9752.58,0.311244,lin_reg,-1560.86,0.043965,0.999527,True,0.43654,42.0
7,year,num_stops,state,AZ,8474.29,0.301916,lin_reg,-1560.86,0.043965,0.999517,True,0.43654,42.0
8,year,num_stops,search_conducted_false_search_conducted_true_d...,6,7240.54,0.197741,lin_reg,-1560.86,0.043965,0.999504,True,0.43654,42.0
9,year,num_stops,search_conducted_true_num_stops_dpgmm,2,6626.86,0.114329,lin_reg,-1560.86,0.043965,0.999496,True,0.43654,42.0


In [28]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh_0.2','SP_thresh0.2').head()

Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_quality', 'trend_type', 'agg_trend',
       'agg_trend_quality', 'distance', 'SP_thresh0.2', 'mean_view_distance',
       'sum_view_SP_thresh0.2'],
      dtype='object')


KeyError: 'sum_view_SP_thresh_0.2'